# Performance Evaluation -- Concurrent Places

In this notebook, we report on some experimental results obtained with Kong.

### Setup

Import librairies.

In [ ]:
%matplotlib inline

import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib as mpl
import numpy as np
import pandas as pd

Path to data.

In [ ]:
path_data = '../../csv/'
path_lists = '../../instances/MCC2021_lists/'

Settings.

In [ ]:
TIMEOUT = 900

## Load Data Frames

### Reductions data frame.

In [ ]:
# Read 'reductions.csv'
df_reductions = pd.read_csv(path_data + 'reductions.csv')
df_reductions = df_reductions.rename(columns={"RATIO_TFG": "RATIO"})[['INSTANCE', 'RATIO']]
df_reductions

### Complete computations data frame (global timeout of 900 seconds).

In [ ]:
# Read 'complete_computations.csv'
df_complete_computations = pd.read_csv(path_data + 'complete_concurrent_computations.csv')
df_complete_computations

### Partial computations data frame (timeout of 60 seconds for the BDD exploration and global timeout of 900 seconds).

In [ ]:
# Read 'partial_computations.csv'
df_partial_computations = pd.read_csv(path_data + 'partial_concurrent_computations.csv')
df_partial_computations

## Tool Confidence rate

### Reliability

Complete computations.

In [ ]:
reliability_complete_matrices = (1 - df_complete_computations.query('CORRECTNESS == False').shape[0] / df_complete_computations.shape[0]) * 100
print('{}%'.format(math.floor(reliability_complete_matrices)))

Partial computations.

In [ ]:
reliability_partial_matrices = (1 - df_partial_computations.query('CORRECTNESS == False').shape[0] / df_partial_computations.shape[0]) * 100
print('{}%'.format(math.floor(reliability_partial_matrices)))

### Correct Matrices

Number of correct and complete matrices.

In [ ]:
df_complete_computations.query('CORRECTNESS == True and TIME_KONG == TIME_KONG').shape[0]

### Summary Table

Computed matrices using Kong.

In [ ]:
computed_matrices_using_Kong = df_complete_computations.query('TIME_KONG == TIME_KONG').shape[0]
computed_matrices_using_Kong

Computed matrices using Caesar.BDD.

In [ ]:
computed_matrices_using_caesar = df_complete_computations.query('TIME_CAESAR == TIME_CAESAR').shape[0]
computed_matrices_using_caesar

In [ ]:
summary = {'Reliability': [reliability_complete_matrices, np.nan], 'Computed Matrices': [computed_matrices_using_Kong, computed_matrices_using_caesar]}
pd.DataFrame(data=summary, index=['Kong', 'Caesar.BDD'])

## Analysis

### General Performance Overview

In [ ]:
# Get computations with the corresponding reduction ratio
df_computations_with_ratio = df_complete_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE')

In [ ]:
def information_per_reduction_range(ratio_min, ratio_max):
    """ Return summary information for a given reduction range.
    """
    df = df_computations_with_ratio.query('RATIO >= {} and RATIO < {}'.format(ratio_min, ratio_max))

    reduction_range = '{}-{}%'.format(ratio_min, ratio_max)

    number_instances = df[['INSTANCE']].shape[0]
    
    computed_matrices_using_Kong = df.query('TIME_KONG == TIME_KONG').shape[0]
    computed_matrices_using_caesar = df.query('TIME_CAESAR == TIME_CAESAR').shape[0]
    
    gain = (1 - computed_matrices_using_caesar / computed_matrices_using_Kong) * 100
    
    return [reduction_range, number_instances, computed_matrices_using_Kong, computed_matrices_using_caesar, gain]

Summary table.

In [ ]:
performance_overview = pd.DataFrame([information_per_reduction_range(ratio_min, ratio_max) for ratio_min, ratio_max in [[1,101], [1,25],[25, 50], [50, 101], [100,101]]], columns=['Reduction Ratio', 'Number of Instances', 'Computed Matrices using Kong', 'Computed Matrices using Caesar.bdd', 'Gain (%)'])
performance_overview.set_index('Reduction Ratio')

### Matrix Computation Times: With VS Without Reduction

Comparison of the number of computed matrices in a limited time (900 seconds) between Kong and Caesar.BDD.

In [ ]:
df = df_complete_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE')
df = df.replace(0, 0.1)

table = df.sort_index()

In [ ]:
def draw_time_vs_matrices_figure(table, ratio_min):
    """ Compare the number of computed matrices between Caesar.BDD and Kong for a given time limit.
    """
    fontsize = 24

    markers = {"TIME_KONG": "s", "TIME_CAESAR": "o"}
    colors = {"TIME_KONG": "tab:orange", "TIME_CAESAR": "tab:blue"}
    labels = {"TIME_KONG": "Kong", "TIME_CAESAR": "Caesar.BDD"}

    plt.figure(figsize=(12, 10))

    for tool in ["TIME_CAESAR", "TIME_KONG"]:
        results = table.query('RATIO >= {}'.format(ratio_min))[tool].dropna().sort_values().reset_index(drop=True)
        results.plot(label=labels[tool], color=colors[tool], linewidth=4, markersize=14, markevery=30)

    plt.yscale('log')
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    ax = plt.gca()
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

    plt.grid()
    plt.legend(loc='upper left', fontsize=fontsize)
    plt.xlabel('Number of computed queries', fontsize=fontsize)
    plt.ylabel('Running time (s)', fontsize=fontsize)
    plt.savefig("complete_matrices_ratio_min_{}.png".format(ratio_min), bbox_inches='tight')
    plt.show()

In [ ]:
draw_time_vs_matrices_figure(table, 0)

In [ ]:
draw_time_vs_matrices_figure(table, 50)

In [ ]:
def matrix_computation_times(ratio_min, ratio_max):
    """ Compare the matrix computation times with vs without reduction.
    """
    fontsize = 23

    # Get computed matrices for a given reduction range (remove instances that timeout using Kong and Caesar.BDD)
    df = df_complete_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and (TIME_KONG == TIME_KONG or TIME_CAESAR == TIME_CAESAR)'.format(25, 49))
    
    # Replace timeout NaN values by the timeout value
    df.loc[df.TIME_KONG != df.TIME_KONG, 'TIME_KONG'] = TIMEOUT
    df.loc[df.TIME_CAESAR != df.TIME_CAESAR, 'TIME_CAESAR'] = TIMEOUT

    # Replace 0 values by 0.1
    df.loc[df.TIME_KONG == 0, 'TIME_KONG'] = 0.01
    df.loc[df.TIME_CAESAR == 0, 'TIME_CAESAR'] = 0.01

    # Get times using Kong and Caesar.BDD 
    x = df['TIME_CAESAR'].to_numpy()
    y = df['TIME_KONG'].to_numpy()

    # Plot time with vs without reduction with a log scale
    plt.figure(figsize=(12,12))
    ax = plt.gca()
    plt.scatter(x=x, y=y, marker='+', s=100, color='orange')

    df = df_complete_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and (TIME_KONG == TIME_KONG or TIME_CAESAR == TIME_CAESAR)'.format(50, 100))
    
    # Replace timeout NaN values by the timeout value
    df.loc[df.TIME_KONG != df.TIME_KONG, 'TIME_KONG'] = TIMEOUT
    df.loc[df.TIME_CAESAR != df.TIME_CAESAR, 'TIME_CAESAR'] = TIMEOUT

    # Replace 0 values by 0.1
    df.loc[df.TIME_KONG == 0, 'TIME_KONG'] = 0.01
    df.loc[df.TIME_CAESAR == 0, 'TIME_CAESAR'] = 0.01

    # Get times using Kong and Caesar.BDD 
    x = df['TIME_CAESAR'].to_numpy()
    y = df['TIME_KONG'].to_numpy()
    
    plt.scatter(x=x, y=y, marker='+', s=100, color='blue')
    
    plt.plot(np.linspace(0.00, TIMEOUT), np.linspace(0.00, TIMEOUT), color='black', linestyle='--', lw=2, scalex=False, scaley=False)
    plt.plot(np.linspace(0.00, TIMEOUT), 0.1 * np.linspace(0.00, TIMEOUT), color='black', linestyle=':', lw=2, scalex=False, scaley=False)
    plt.plot(np.linspace(0.00, TIMEOUT), 0.01 * np.linspace(0.00, TIMEOUT), color='black', linestyle=':', lw=2, scalex=False, scaley=False)
    ax.set_xscale('log')
    ax.set_yscale('log')
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.xlabel('Computation time using Caesar.bdd (without reduction) (s)', fontsize=fontsize)
    plt.ylabel('Computation time using Kong (with reduction) (s)', fontsize=fontsize)
    plt.savefig("pics/time_{}_{}.png".format(ratio_min, ratio_max), bbox_inches = 'tight')
    plt.show()

In [ ]:
matrix_computation_times(25, 100)

In [ ]:
matrix_computation_times(50, 100)

### Partial Concurrency Matrices: With VS Without Reduction

In [ ]:
# Drop instances that take more than 600 seconds using Kong or Caesar.BDD (outliers).
df_partial_computations_without_outliers = df_partial_computations.query('TIME_KONG < 600 and TIME_CAESAR < 600')

Comparison of the filling ratio with and without reduction (60 seconds of timeout for the BDD exploration).

In [ ]:
def filling_ratio(ratio_min, ratio_max):
    """ Plot the filling ratio of the matrices with vs without reduction.
    """
    fontsize = 23

    # Get computed relations for a given reduction range (remove instances that timeout or that are fully computed with both tools)
    df = df_partial_computations_without_outliers.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and (NUMBER_RELATIONS_KONG > 0 or NUMBER_RELATIONS_CAESAR > 0) and (NUMBER_RELATIONS_KONG < RELATION_SIZE or NUMBER_RELATIONS_CAESAR < RELATION_SIZE)'.format(50, 100))
    
    # Get filling ratios using Kong and Caesar.BDD
    x = 100 * df['NUMBER_RELATIONS_CAESAR'].to_numpy() / df['RELATION_SIZE'].to_numpy()
    y = 100 * df['NUMBER_RELATIONS_KONG'].to_numpy() / df['RELATION_SIZE'].to_numpy()

    # Plot ratios with vs without reduction
    plt.figure(figsize=(12,12))
    ax = plt.gca()
    ax.set_aspect('equal')
    plt.scatter(x=x, y=y, marker='+', linewidths=3, s=150, color='tab:blue')  

    # Get computed relations for a given reduction range (remove instances that timeout or that are fully computed with both tools)
    df = df_partial_computations_without_outliers.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and (NUMBER_RELATIONS_KONG > 0 or NUMBER_RELATIONS_CAESAR > 0) and (NUMBER_RELATIONS_KONG < RELATION_SIZE or NUMBER_RELATIONS_CAESAR < RELATION_SIZE)'.format(25, 49))
    
    # Get filling ratios using Kong and Caesar.BDD
    x = 100 * df['NUMBER_RELATIONS_CAESAR'].to_numpy() / df['RELATION_SIZE'].to_numpy()
    y = 100 * df['NUMBER_RELATIONS_KONG'].to_numpy() / df['RELATION_SIZE'].to_numpy()
    
    plt.scatter(x=x, y=y, marker='+', linewidths=3, s=150, color='tab:orange')  
    ## To remove end
    plt.plot(np.linspace(0, 100), np.linspace(0, 100), color='gray', linestyle='--', lw=2, scalex=False, scaley=False)
    plt.grid()
    plt.xlim(-5,105)
    plt.ylim(-5,105)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.xlabel('Filling ratio using Caesar.bdd (without reduction) (%)', fontsize=fontsize)
    plt.ylabel('Filling ratio using Kong (with reduction) (%)', fontsize=fontsize)
    plt.savefig("pics/filling_ratio_{}_{}.png".format(ratio_min, ratio_max), bbox_inches = 'tight')
    plt.show()

In [ ]:
filling_ratio(25, 100)

Median computation time per instance with Kong.

In [ ]:
df_partial_computations_without_outliers['TIME_KONG'].median()

Mean computation time per instance with Kong.

In [ ]:
df_partial_computations_without_outliers['TIME_KONG'].mean()

Median computation time per instance with Caesar.BDD.

In [ ]:
df_partial_computations_without_outliers['TIME_CAESAR'].median()

Median computation time per instance with Caesar.BDD.

In [ ]:
df_partial_computations_without_outliers['TIME_CAESAR'].mean()

### Concurrent and Independent Places: With vs Without Reduction

Comparison of the filling ratio for concurrent places and independent places (independently) with vs without reduction.

In [ ]:
def concurrent_and_independant_places_filling_ratio(ratio_min, ratio_max):
    """ Compare the filling ratio for concurrent and independent places with vs without reduction.
    """
    fontsize = 23

    # Get computed relations for a given reduction range (remove instances that timeout or that are fully computed with both tools)
    df = df_partial_computations_without_outliers.drop(['RELATION_SIZE', 'TIME_KONG', 'TIME_CAESAR', 'CORRECTNESS'], axis=1).join(df_reductions.set_index('INSTANCE'), on='INSTANCE').join(df_complete_computations.drop(['TIME_KONG', 'TIME_CAESAR', 'CORRECTNESS'], axis=1).set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and RELATION_SIZE > 0 and (NUMBER_RELATIONS_KONG > 0 or NUMBER_RELATIONS_CAESAR > 0) and (NUMBER_RELATIONS_KONG < RELATION_SIZE or NUMBER_RELATIONS_CAESAR < RELATION_SIZE)'.format(ratio_min, ratio_max))

    # Get filling ratios for concurrent places ('1') using Kong and Caesar.BDD
    x = 100 * df['CONCURRENT_PLACES_CAESAR'].to_numpy() / df['CONCURRENT_PLACES'].to_numpy()
    y = 100 * df['CONCURRENT_PLACES_KONG'].to_numpy() / df['CONCURRENT_PLACES'].to_numpy()

    # Plot ratios with vs without reduction
    plt.figure(figsize=(12,12))
    ax = plt.gca()
    plt.scatter(x=x, y=y, marker='+', linewidths=3, s=150)  
    plt.plot(np.linspace(0, 500), np.linspace(0, 500), color='black', linestyle='--', lw=2, scalex=False, scaley=False)
    plt.xlim(-0.1,101)
    plt.ylim(-0.1,101)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.xlabel('Filling ratio of the concurrent places without reduction (%)', fontsize=fontsize)
    plt.ylabel('Filling ratio of the concurrent places with reduction (%)', fontsize=fontsize)
    plt.savefig("pics/concurrent_places_{}_{}.png".format(ratio_min, ratio_max), bbox_inches = 'tight')
    plt.show()

    # Get filling ratios for independent places ('0') using Kong and Caesar.BDD
    x = 100 * (df['NUMBER_RELATIONS_CAESAR'].to_numpy() - df['CONCURRENT_PLACES_CAESAR'].to_numpy()) / (df['RELATION_SIZE'].to_numpy() - df['CONCURRENT_PLACES'].to_numpy())
    y = 100 * (df['NUMBER_RELATIONS_KONG'].to_numpy() - df['CONCURRENT_PLACES_KONG'].to_numpy()) / (df['RELATION_SIZE'].to_numpy() - df['CONCURRENT_PLACES'].to_numpy())

    # Plot filling ratios with vs without reduction
    plt.figure(figsize=(12,12))
    ax = plt.gca()
    plt.scatter(x=x, y=y, marker='+', linewidths=3, s=150)  
    plt.plot(np.linspace(0, 500), np.linspace(0, 500), color='black', linestyle='--', lw=2, scalex=False, scaley=False)
    plt.xlim(-0.1,101)
    plt.ylim(-0.1,101)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.xlabel('Filling ratio of the independent places without reduction (%)', fontsize=fontsize)
    plt.ylabel('Filling ratio of the independent places with reduction (%)', fontsize=fontsize)
    plt.savefig("pics/independent_places_{}_{}.png".format(ratio_min, ratio_max), bbox_inches = 'tight')
    plt.show()

In [ ]:
concurrent_and_independant_places_filling_ratio(25, 100)